In [ ]:
!pip install pandas
!pip install numpy
!pip install ffn

In [42]:
import pandas as pd 
import numpy as np 
import ffn
#取樣週期，MS = 月 , W=周 , D =日
sample = 'W'
#參考大盤指數，^GSPC,^Twii,^DJI,^IXIC
index_name = "^twii"
#資產輸入，需要至yahoo finance查詢代碼 (台灣上市+.TW，上櫃加.TWO)
stock_list = ["2454.TW","2330.TW","1512.TW"]
#開始時間
start_time = "2016-01-01"


# 下方不需變動

In [43]:
#名稱前處理
stock_high_list = ""
stock_low_list = ""
stock_close = ""

for stock in stock_list :
    stock_high = stock + ":High"
    stock_low  = stock + ":Low"
    stock_high_list = stock_high_list +","+ stock_high
    stock_low_list = stock_low_list +","+ stock_low
    stock_close = stock_close +","+ stock  
    
#抓取資產開高低收
close = ffn.get(stock_close[1:] +"," +index_name,start = start_time) 
high = ffn.get(stock_high_list[1:] +"," +index_name,start = start_time) 
low = ffn.get(stock_low_list[1:] +"," +index_name,start = start_time) 
interest_rate = ffn.get("^tnx",start = start_time) 

#取樣週期調整利率參數
if sample.upper()  == 'MS' :
    r_div = 12
elif sample.upper() == "W" :
    r_div = 52
elif sample.upper() == "D" :
    r_div = 365
    
#column名稱前處理
non_index_stock_list = stock_list
index = index_name[1:]
stock_list.append(index)
#抽樣頻率調整資產
Month_first = close.resample(sample).first()
Month_max =  high.resample(sample).max()
Month_low =  low.resample(sample).min()
Month_max.columns = stock_list
Month_low.columns = stock_list
r = interest_rate.resample(sample).mean() / 100 / r_div
#Month_max =  prices.groupby([prices.index.year,prices.index.month]).agg(max)

#計算pi
pi = pd.DataFrame(np.nan, index=Month_first.index ,columns = ["pi"])
pi = (Month_first[index]*(1+r["tnx"]) - Month_low[index]) / (Month_max[index] - Month_low[index])

#各資產計算並儲存
stock_price = {} 
for stock in non_index_stock_list :
    stock_price[stock] = (pi * Month_max[stock]) + ((1-pi)*Month_low[stock]) /(1+r["tnx"])
stock_price = pd.DataFrame(stock_price).dropna()
stock_price.to_csv("stock_prices.csv")
print(stock_price)

               2454.TW     2330.TW   1512.TW          twii
Date                                                      
2016-01-10  252.652957  143.670197  7.399458   8117.690385
2016-01-17  205.294476  134.945264  6.119660   7790.264489
2016-01-24  216.903152  137.423764  5.607677   7813.553483
2016-01-31  205.658140  138.593197  5.258199   7894.958632
2016-02-07  217.225877  146.143201  5.203519   8159.977369
...                ...         ...       ...           ...
2022-11-20  694.843758  442.647701  3.158896  14175.195056
2022-11-27  703.536099  476.352286  3.155268  14449.720541
2022-12-04  695.887483  473.458478  3.216517  14557.104540
2022-12-11  756.195302  498.229725  3.638577  14991.101201
2022-12-18  684.331169  472.687632  4.006159  14617.169937

[357 rows x 4 columns]
